In [42]:
#HSV segmentation

In [43]:
%matplotlib inline
from matplotlib import pyplot as plt
from skimage.transform import resize
from skimage import data,color, exposure,feature,io
from math import sqrt
from skimage.color import rgb2gray
import skimage.exposure as imexp
from skimage.morphology import binary_opening,disk
from skimage.filters import gabor_kernel
from PIL import Image
from scipy import ndimage as ndi
from skimage.feature import greycomatrix, greycoprops
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

from skimage import img_as_float
from skimage.morphology import reconstruction


import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from matplotlib import style

style.use('ggplot')

In [44]:
#loading data

In [45]:


imgSamples = [];
imgHsvSamples = [];


patches = [];
patchesHsv = [];


whitePatches = [];
whitePatchesHsv = [];

blackPatches = [];
blackPatchesHsv = [];


lesionPatches = [];
lesionPatchesHsv = [];


samplingSize =(300,300)
#samplingMode = 'nearest'
samplingMode = 'wrap'
WINDOW_FRAME = 2
gausssianSig = 0.4


for i in range(0,15):
    temp1= resize(np.array(Image.open('newSamples/'+str(i+1)+'.jpg')), samplingSize,mode=samplingMode)
    image1 = img_as_float(temp1)
    image1 = gaussian_filter(image1,gausssianSig)
    imgSamples.append(image1)
                      
for i in range(len(imgSamples)):
    imgHsvSamples.append(color.rgb2hsv(imgSamples[i]))
    
    
for i in range(0,1):
    
    temp2= np.array(Image.open('newPatches4/normal'+str(i+1)+'.jpg'))
    image2 = img_as_float(temp2)
    image2 = gaussian_filter(image2, gausssianSig)
    patches.append(image2)
                      
for i in range(len(patches)):
    patchesHsv.append(color.rgb2hsv(patches[i]))
    
for i in range(0,1):
    
        
    temp3= np.array(Image.open('newPatches4/white'+str(i+1)+'.jpg'))
    image3 = img_as_float(temp3)
    image3 = gaussian_filter(image3, gausssianSig)
    whitePatches.append(image3)
                      
for i in range(len(whitePatches)):
    whitePatchesHsv.append(color.rgb2hsv(whitePatches[i]))

for i in range(0,1):
    
    temp4= np.array(Image.open('newPatches4/black'+str(i+1)+'.jpg'))
    image4 = img_as_float(temp4)
    image4 = gaussian_filter(image4,gausssianSig)
    blackPatches.append(image4)
    
for i in range(len(blackPatches)):
    blackPatchesHsv.append(color.rgb2hsv(blackPatches[i]))
    
for i in range(0,3):
    
    temp5=np.array(Image.open('newPatches4/lesion'+str(i+1)+'.jpg'))
    image5 = img_as_float(temp5)
    image5 = gaussian_filter(image5, gausssianSig)
    lesionPatches.append(temp5)
    
for i in range(len(lesionPatches)):
    lesionPatchesHsv.append(color.rgb2hsv(lesionPatches[i]))


print(len(imgHsvSamples))  
print(len(patchesHsv)) 
print(len(whitePatchesHsv)) 
print(len(blackPatchesHsv)) 
print(len(lesionPatches)) 
print(imgHsvSamples[0].shape)
print(patchesHsv[0].shape)
print(whitePatchesHsv[0].shape)
print(blackPatchesHsv[0].shape)
print(lesionPatchesHsv[0].shape)

15
1
1
1
3
(300, 300, 3)
(100, 100, 3)
(20, 20, 3)
(20, 20, 3)
(25, 25, 3)


In [46]:
##saturation thresholding

In [47]:
def satThreshold(v,s):    ##return hue or intensity as dominant feature
    th = 1.0 - 0.8*v;
    if(s>th):
        return "h"
    else: 
        return "v"
    

In [48]:
def approximate(img):
    tmpImg = np.zeros(img.shape)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            dominantVal = satThreshold(img[i,j,2],img[i,j,1])
#             print("dominantVal----"+dominantVal)
            tmpImg[i,j,:] = img[i,j,:]
            if dominantVal == "h":
                tmpImg[i,j,0] = img[i,j,0]
                tmpImg[i,j,1] = 1.0
                tmpImg[i,j,2] = 1.0
                
            else:
                tmpImg[i,j,0] = 1.0
                tmpImg[i,j,1] = 1.0
                tmpImg[i,j,2] = img[i,j,2]
        
    return tmpImg

In [49]:
###Gabor filter

def filterImage(image, kernels):
    filtered = []
    for k, kernel in enumerate(kernels):
        filtered.append(ndi.convolve(image, kernel, mode='wrap'))
    return filtered

In [50]:

def mass_compute_texture_feat(img):

    imgToFilter = rgb2gray(img)
    filteredImg = filterImage(imgToFilter,kernels)
    print "filteredImg"
    print len(filteredImg)
    
    gaborEachFeats = 5
    gaborFeatsLen = len(filteredImg)*gaborEachFeats
    glcmFeats=2
    dim = gaborFeatsLen+glcmFeats
    
    feats = np.zeros((img.shape[0],img.shape[1],dim))
    step = 0

    for m_index,image in enumerate(filteredImg):
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = image[max(i-WINDOW_FRAME,0):min(i+WINDOW_FRAME,image.shape[0]-1),max(j-WINDOW_FRAME,0):min(j+WINDOW_FRAME,image.shape[1]-1)]
                
                feats[i,j, step] = np.sum(window**2)
                feats[i,j, step+1] =  np.sum(np.absolute(window))
                feats[i,j, step+2] = image[i,j]
                feats[i,j, step+3] = window.mean()
                feats[i,j, step+4] = window.var()
                
        step += gaborEachFeats
    
    
    for i in range(imgToFilter.shape[0]):
        for j in range(imgToFilter.shape[1]):
                window = imgToFilter[max(i-WINDOW_FRAME,0):min(i+WINDOW_FRAME,imgToFilter.shape[0]-1),max(j-WINDOW_FRAME,0):min(j+WINDOW_FRAME,imgToFilter.shape[1]-1)]
                glcm = greycomatrix(window, [5], [0], 256, symmetric=True, normed=True)
                feats[i,j, gaborFeatsLen] = greycoprops(glcm, 'dissimilarity')[0, 0]
                feats[i,j, gaborFeatsLen+1] = greycoprops(glcm, 'correlation')[0, 0]
    return feats

In [51]:

def ref_img_feats(patchesName):
    
    
    if patchesName == 'normal':   
        targetImg = patches
        targetImgHSV = patchesHsv
    elif patchesName == 'white':
        targetImg = whitePatches
        targetImgHSV = whitePatchesHsv
    elif patchesName == 'black':
        targetImg = blackPatches
        targetImgHSV = blackPatchesHsv
    elif patchesName == 'lesion':
        targetImg = lesionPatches
        targetImgHSV = lesionPatchesHsv
        
    
    ref_feats = {}

    
    for i in range(len(targetImgHSV)):
       
        ref_feats[i] = mass_compute_texture_feat(targetImg[i])
            
        ref_feats[i] = np.mean(ref_feats[i], axis=0)
        
#     print "ref_feats"
#     print ref_feats
    return ref_feats

In [52]:
###k-means for features


####k-means class



class K_Means_Feats:
    def __init__(self, k=10, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        self.rgbColors = np.array([[0,255,0],[255,255,255],[0,0,0],[255,0,0],[0,0,255],[255,255,0],[0,255,255],[255,0,255],[51,51,255],[102,102,0],[255,0,127],[160,32,240],[238,130,238]])
        self.rgbColors.astype(float)
#        white, green, ,red, yellow, purple,violet



    
    
    def fit(self,img,which=1):

        self.centroids = {}
        
        tempImg = img

        p_index = len(patches)
        w_p_index = len(patches)+len(whitePatches)
        b_p_index = len(patches)+len(whitePatches)+len(blackPatches)
        
        
        
        
        ref_feats_n = ref_feats_dict[which-1][0]
        ref_feats_w = ref_feats_dict[which-1][1]
        ref_feats_b = ref_feats_dict[which-1][2]
        ref_feats_l = ref_feats_dict[which-1][3]
        
        
        #centroids for normal parts of the esophagus
        for i in range(len(patches)):
            self.centroids[i] = ref_feats_n[i][0]
        #centroids for white and bright parts
        for i in range(len(whitePatches)):
            self.centroids[p_index+i] = ref_feats_w[i][0]
        #centroids for black parts
        for i in range(len(blackPatches)):
            self.centroids[w_p_index+i] = ref_feats_b[i][0] 
            
        for i in range(len(lesionPatches)):
            self.centroids[b_p_index+i] = ref_feats_l[i][0]     
        #ramdomly select centroid for the lesions   
#         self.centroids[self.k-1] = img[0,0,:]


#         print "centroids----"
#         print self.centroids[0]
#         print self.centroids
      

        for i in range(self.max_iter):
            self.clusters = {}
            self.clustersIndeces = {}
                ##classes holder
            for i in range(self.k):
                self.clusters[i] = []
                self.clustersIndeces[i] = []
        
            for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    distances = [np.linalg.norm(img[i,j]-self.centroids[centroid]) for centroid in self.centroids]
                    clust_index = distances.index(min(distances))
                    self.clustersIndeces[clust_index].append([i,j])
                    self.clusters[clust_index].append(img[i,j])
                    
            prev_centroids = dict(self.centroids)         
            
            ##  re-assign centroids 
            for item in self.clusters:
                self.centroids[item] = np.average(self.clusters[item],axis=0)

            optimized = True
            
            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/((original_centroid)*100.0)) > self.tol:
                    optimized = False

            if optimized:
                break
        
        output = np.zeros((img.shape[0],img.shape[1],3),np.uint8); 
        
##assigning colors  

#         print "last self.centroids"
#         print self.centroids

        for cent in self.centroids:
            for pair in self.clustersIndeces[cent]:
                
                if  (cent < p_index): #normal pixels---assign green color
                    output[pair[0],pair[1],0] = self.rgbColors[0][0]
                    output[pair[0],pair[1],1] = self.rgbColors[0][1]
                    output[pair[0],pair[1],2] = self.rgbColors[0][2]
                    
                elif  (cent >= p_index) & (cent < w_p_index): #white pixels----assign white color
                    output[pair[0],pair[1],0] = self.rgbColors[1][0]
                    output[pair[0],pair[1],1] = self.rgbColors[1][1]
                    output[pair[0],pair[1],2] = self.rgbColors[1][2]
                    
                elif  (cent >= w_p_index) & (cent < b_p_index): # black pixels---assign black color
                    output[pair[0],pair[1],0] = self.rgbColors[2][0]
                    output[pair[0],pair[1],1] = self.rgbColors[2][1]
                    output[pair[0],pair[1],2] = self.rgbColors[2][2] 
                    
                elif  (cent >= b_p_index): #lesions------assign red color
                    output[pair[0],pair[1],0] = self.rgbColors[3][0]
                    output[pair[0],pair[1],1] = self.rgbColors[3][1]
                    output[pair[0],pair[1],2] = self.rgbColors[3][2] 
                    
        return output
    
    
    
    
    
    
    
    
    
    
    def fit_and_filter(self,img):

        self.centroids = {}
        
        tempImg = img

        p_index = len(patches)
        w_p_index = len(patches)+len(whitePatches)
        b_p_index = len(patches)+len(whitePatches)+len(blackPatches)

        ref_feats_n = ref_feats_normal
        ref_feats_w = ref_feats_white
        ref_feats_b = ref_feats_black
        ref_feats_l = ref_feats_lesion
            
            
        
        print('indeces----')
        print p_index
        print w_p_index
        
        for i in range(len(patches)):
            self.centroids[i] = ref_feats_n[i][0]
        #centroids for white and bright parts
        for i in range(len(whitePatches)):
            self.centroids[p_index+i] = ref_feats_w[i][0]
        #centroids for black parts
        for i in range(len(blackPatches)):
            self.centroids[w_p_index+i] = ref_feats_b[i][0] 
            
        for i in range(len(lesionPatches)):
            self.centroids[b_p_index+i] = ref_feats_l[i][0]     
            

        for i in range(self.max_iter):
            self.clusters = {}
            self.clustersIndeces = {}
                ##classes holder
            for i in range(self.k):
                self.clusters[i] = []
                self.clustersIndeces[i] = []
        
            for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    distances = [np.linalg.norm(img[i,j]-self.centroids[centroid]) for centroid in self.centroids]
                    clust_index = distances.index(min(distances))
                    self.clustersIndeces[clust_index].append([i,j])
                    self.clusters[clust_index].append(img[i,j])
                    
            prev_centroids = dict(self.centroids)         
            
            ##  re-assign centroids 
            for item in self.clusters:
                self.centroids[item] = np.average(self.clusters[item],axis=0)

            optimized = True
            
            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/((original_centroid)*100.0)) > self.tol:
                    optimized = False

            if optimized:
                break
        
        output = np.zeros((img.shape[0],img.shape[1]),np.uint8); 
        
##assigning colors  

        for cent in self.centroids:
            for pair in self.clustersIndeces[cent]: 
                if  (cent >= b_p_index): #lesions------assign red color
                    output[pair[0],pair[1]] = 255
                    
        return binary_opening(binary_opening(output, disk(3)),disk(3))


In [53]:
# preparing Gabor filter bank kernels
kernels = []
for theta in np.arange(0, np.pi, np.pi / 6):
    for sigma in (1.0,1.5):  #gausian kernel window size
#         for frequency in (0.1, 0.2):
            kernel = np.real(gabor_kernel(0.15, theta=theta,sigma_x=sigma, sigma_y=sigma))
            kernels.append(kernel)

In [54]:
ref_feats_dict = {}

i = 0
ref_feats_dict[i]=[]
ref_feats_dict[i].append(ref_img_feats('normal'))
ref_feats_dict[i].append(ref_img_feats('white'))
ref_feats_dict[i].append(ref_img_feats('black'))
ref_feats_dict[i].append(ref_img_feats('lesion'))

filteredImg
12
filteredImg
12
filteredImg
12
filteredImg
12
filteredImg
12
filteredImg
12


In [ ]:
##Best working ones

In [ ]:
colnum = 3
rownum = len(imgSamples)

fig, axes = plt.subplots(rownum, colnum, figsize=(65, rownum*20), sharex=True, sharey=True)
ax = axes.ravel()
index = 0

# divide into white, black, normal and abonomal parts of the esophagus
kmeanf = K_Means_Feats(len(patches)+len(whitePatches)+len(blackPatches)+len(lesionPatches))

for i in range(0, len(imgHsvSamples)):
    
    ax[index].imshow(imgSamples[i],cmap='gray')
    ax[index].set_title("original rgb "+str(i),fontsize=60)
    ax[index].axis('off') 
   
    ax[index+1].imshow(approximate(imgHsvSamples[i]),cmap='gray')
    ax[index+1].set_title("segmented "+str(i),fontsize=60)
    ax[index+1].axis('off')
    
    ax[index+2].imshow(kmeanf.fit(mass_compute_texture_feat(imgHsvSamples[i]),which=1) ,cmap='gray')
    ax[index+2].set_title("Texture Seg"+' - '+str(i), fontsize=60)
    ax[index+2].axis('off')
    
    index += colnum
    
    
    print "img "+str(i)+"----done"
    
plt.show()

filteredImg
12
